#### Задача 1 (10 баллов). 

Напишем мини-стратегическую игру, в которой игроку предлагается управлять городом (SimCity видели когда-нибудь?) У нас есть собственно класс "город", внутри города есть постройки разных типов, которые мы, как градоправитель, строим. Также у города есть атрибуты: уровень счастья жителей, уровень экологии, образования, миграции и так далее (на что вашей фантазии хватит). Постройки к этим атрибутам добавляют и прибавляют баллы. Можно завести тип постройки, которая на каждом ходе игры приносит в городской бюджет деньги (налоговая служба?.. заводы?). Постройки необходимо реализовать с помощью наследования, у некоторых построек могут быть свои методы, например, больница может устраивать медосмотры и вакцинации и поднимать этим уровень благосостояния жителей, а на заводе с рандомной вероятностью может случиться авария, которая понизит экологию. Не забудьте реализовать головной класс "игра", в котором будет метод play, можно в нем хранить и городской бюджет. 

Фантазия приветствуется: чем оригинальнее будет ваша игра, тем выше шанс, что я поставлю дополнительный балл. 

In [82]:
import random


class Building:
    def __init__(self, name, cost):
        self.name = name
        self.cost = cost
    
    def build_effects(self, city):
        city.budget -= self.cost
        print(f'Вы построили {self.name}. Стоимость: {self.cost}')

    def update(self, city):
        pass


class IndustrialBuilding(Building):
    def build_effects(self, city):
        super().build_effects(city)
        city.unemployment -= 5
        city.ecology -= 5


class Factory(IndustrialBuilding):        
    def update(self, city):
        city.budget += 500
        city.ecology -= 0.5
        if random.random() < 0.05:
            self.accident(city)
    
    def accident(self, city):
        city.ecology -= 10
        city.happiness -= 5
        city.budget -= 500
        print(f'На заводе произошла авария!')


class SocialBuilding(Building):
    def build_effects(self, city):
        super().build_effects(city)
        city.happiness += 5


class Hospital(SocialBuilding):
    def build_effects(self, city):
        super().build_effects(city)
        city.health += 10
    
    def update(self, city):
        city.health += 0.5
        city.budget -= 100


class School(SocialBuilding):
    def build_effects(self, city):
        super().build_effects(city)
        city.education += 5

    def update(self, city):
        city.education += 0.3
        city.budget -= 50


class University(SocialBuilding):
    def build_effects(self, city):
        super().build_effects(city)
        city.education += 10
        city.unemployment -= 2

    def update(self, city):
        city.education += 0.5
        city.unemployment -= 0.1
        city.budget -= 200


class ResidentialBuilding(Building):
    def build_effects(self, city):
        super().build_effects(city)
        city.population += 100
        city.happiness += 2


class GreenBuilding(Building):
    def build_effects(self, city):
        super().build_effects(city)
        city.ecology += 10 


class Park(GreenBuilding):
    def build_effects(self, city):
        super().build_effects(city)
        city.happiness += 5

    def update(self, city):
        city.ecology += 0.5
        city.happiness += 0.2
        city.budget -= 20


class SolarPlant(GreenBuilding):
    def update(self, city):
        city.budget += 100
        city.ecology += 0.2


class City:
    def __init__(self, name):
        self.name = name
        self.population = 1000
        self.buildings = []
        self.budget = 15000
        self.happiness = 50
        self.health = 50
        self.education = 50
        self.ecology = 50
        self.unemployment = 5

    def add_building(self, building):
        self.buildings.append(building)
        building.build_effects(self)

    def update(self):
        self.happiness -= 0.5
        self.health -= 0.3
        self.education -= 0.2
        self.ecology -= 0.1
        self.unemployment += 0.2
        self.budget += self.population * 0.1 # налоги

        for building in self.buildings:
            building.update(self)

        self.happiness = max(0, min(100, self.happiness))
        self.health = max(0, min(100, self.health))
        self.education = max(0, min(100, self.education))
        self.ecology = max(0, min(100, self.ecology))
        self.unemployment = max(0, min(100, self.unemployment))

        # рост населения
        growth_rate = (self.happiness + self.health + self.education) / 300 - self.unemployment / 100
        self.population += int(self.population * growth_rate * 0.01)


class Game:
    def __init__(self):
        city_name = input('Введите название вашего города: ')
        self.city = City(city_name if city_name else 'Новый город')
        self.turn = 0
        self.max_turns = 50

    def run(self):
        while self.turn < self.max_turns:
            self.turn += 1
            print(f'\nХод {self.turn}')

            self.city_status()
            if self.player_action() == 'exit':
                break

            self.city.update()
            self.random_event()

            if self.check_game_over():
                break

        else:
            print('\nДостигнуто максимальное количество ходов.')

        self.end_game()

    def city_status(self):
        print(f'Население: {self.city.population}')
        print(f'Бюджет: {self.city.budget:.2f}')
        print(f'Счастье: {self.city.happiness:.2f}%')
        print(f'Здоровье: {self.city.health:.2f}%')
        print(f'Образование: {self.city.education:.2f}%')
        print(f'Безработица: {self.city.unemployment:.2f}%')
        print(f'Экология: {self.city.ecology:.2f}%')

    def player_action(self):
        print('\nВыберите действие:')
        print('1. Построить здание')
        print('2. Специальное действие')
        print('3. Пропустить ход')
        print('4. Завершить игру')
        
        choice = input('Ваш выбор: ')
        
        if choice == '1':
            self.build()
        elif choice == '2':
            self.special_action()
        elif choice == '3':
            print('Вы пропускаете ход.')
        elif choice == '4':
            return 'exit'
        else:
            print('Неверный выбор. Ход пропущен.')

    def build(self):
        print('\nКакое здание вы хотите построить?')
        print('1. Завод (10000)')
        print('2. Больница (15000)')
        print('3. Школа (10000)')
        print('4. Университет (15000)')
        print('5. Жилой дом (5000)')
        print('6. Парк (5000)')
        print('7. Солнечная электростанция (15000)')
        
        choice = input('Ваш выбор: ')
        
        if choice == '1':
            building = Factory('Завод', 10000)
        elif choice == '2':
            building = Hospital('Больница', 15000)
        elif choice == '3':
            building = School('Школа', 10000)
        elif choice == '4':
            building = University('Университет', 15000)
        elif choice == '5':
            building = ResidentialBuilding('Жилой дом', 5000)
        elif choice == '6':
            building = Park('Парк', 5000)
        elif choice == '7':
            building = SolarPlant('Солнечная электростанция', 15000)
        else:
            print('Неверный выбор. Строительство отменено.')
            return
        
        if self.city.budget >= building.cost:
            self.city.add_building(building)
        else:
            print('Недостаточно средств для строительства :(')

    def special_action(self):
        print('\nВыберите действие:')
        print('1. Провести городской фестиваль')
        print('2. Организовать массовую диспансеризацию')

        choice = input('Ваш выбор: ')
        
        if choice == '1':
            self.organize_festival()
        elif choice == '2':
            self.organize_checkup()
        else:
            print('Неверный выбор. Действие отменено.')

    def organize_festival(self):
        cost = int(self.city.population * 0.5) # стоимость зависит от населения
        if self.city.budget >= cost:
            self.city.budget -= cost
            happiness_up = random.randint(3, 7)
            self.city.happiness += happiness_up
            print(f'Фестиваль прошел успешно!')
            print(f'Уровень счастья повысился на {happiness_up}')
            print(f'Бюджет уменьшился на {cost}')
        else:
            print(f'Недостаточно средств для проведения фестиваля.')

    def organize_checkup(self):
        hospitals = [b for b in self.city.buildings if isinstance(b, Hospital)]
        if hospitals:
            cost = int(self.city.population * 0.5) 
            if self.city.budget >= cost:
                self.city.budget -= cost
                health_up = random.randint(3, 7)
                happiness_up = random.randint(1, 3)

                self.city.health += health_up
                self.city.happiness += happiness_up

                print('Массовая диспансеризация проведена успешно!')
                print(f'Уровень здоровья населения повысился на {health_up}.')
                print(f'Уровень счастья повысился на {happiness_up}.')
                print(f'Бюджет уменьшился на {cost}.')
            else:
                print(f'Недостаточно средств для проведения диспансеризации.')
        else:
            print('В городе нет больниц.')
        
    def random_event(self):
        if random.random() < 0.05:
            print('Стихийное бедствие! Экология и счастье снизились.')
            self.city.ecology -= 10
            self.city.happiness -= 10

    def check_game_over(self):
        if self.city.budget < 0:
            print('Ваш город обанкротился!')
            return True
        if self.city.happiness < 5:
            print('Депрессия победила...')
            return True
        if self.city.ecology < 5:
            print('Экологическая катастрофа! Город стал непригодным для жизни.')
            return True
        if self.city.health < 5:
            print('Эпидемия! Система здравоохранения не справляется.')
            return True
        return False

    def end_game(self):
        print('\nИгра завершена.')
        print(f'Вы управляли городом {self.city.name} на протяжении {self.turn} ходов.')
        self.city_status()


game = Game()
game.run()


Ход 1
Население: 1000
Бюджет: 15000.00
Счастье: 50.00%
Здоровье: 50.00%
Образование: 50.00%
Безработица: 5.00%
Экология: 50.00%

Выберите действие:
1. Построить здание
2. Специальное действие
3. Пропустить ход
4. Завершить игру

Какое здание вы хотите построить?
1. Завод (10000)
2. Больница (15000)
3. Школа (10000)
4. Университет (15000)
5. Жилой дом (5000)
6. Парк (5000)
7. Солнечная электростанция (15000)
Вы построили Завод. Стоимость: 10000

Ход 2
Население: 1004
Бюджет: 5600.00
Счастье: 49.50%
Здоровье: 49.70%
Образование: 49.80%
Безработица: 0.20%
Экология: 44.40%

Выберите действие:
1. Построить здание
2. Специальное действие
3. Пропустить ход
4. Завершить игру

Какое здание вы хотите построить?
1. Завод (10000)
2. Больница (15000)
3. Школа (10000)
4. Университет (15000)
5. Жилой дом (5000)
6. Парк (5000)
7. Солнечная электростанция (15000)
Вы построили Парк. Стоимость: 5000

Ход 3
Население: 1009
Бюджет: 1180.40
Счастье: 54.20%
Здоровье: 49.40%
Образование: 49.60%
Безработица: 